In [1]:
import numpy as np
import torch
import pychop as pychop
from pychop.chop import chop
from pychop import float_params
from time import time
from numpy import linalg
import jax
# from pychop.chop import chop
# from pychop.quant import quant
from time import time
from scipy.io import savemat
np.set_printoptions(precision=4)

In [2]:
np.random.seed(0)

X_np = np.random.randn(5000, 5000) # Numpy array
X_th = torch.Tensor(X_np) # torch array
X_jx = jax.numpy.asarray(X_np)
print(X_np)


savemat("tests/verify.mat", {"X":X_np})

[[ 1.7641  0.4002  0.9787 ...  0.9292  0.2294  0.4144]
 [ 0.3097 -0.7375 -1.5369 ...  0.5169 -0.0329  1.2981]
 [-0.2021 -0.8332  1.7336 ...  0.7531 -0.581  -0.1984]
 ...
 [ 1.0743  1.1885  0.5093 ...  0.0705  0.5998 -2.4103]
 [ 0.3243 -0.0234  1.6287 ... -0.1609 -1.5977  1.4147]
 [ 0.6346  1.3809  0.5483 ...  0.3076 -0.1108  0.8386]]


### print unit-roundoff in machine

In [3]:
float_params()

,,u,xmins,xmin,xmax,p,emins,emin,emax
0,q43,6.25e-02,1.95e-03,1.56e-02,2.40e+02,4,-9,-6,7
1,q52,1.25e-01,1.53e-05,6.10e-05,5.73e+04,3,-16,-14,15
2,b,3.91e-03,9.18e-41,1.18e-38,3.39e+38,8,-133,-126,127
3,h,4.88e-04,5.96e-08,6.10e-05,6.55e+04,11,-24,-14,15
4,t,4.88e-04,1.15e-41,1.18e-38,3.40e+38,11,-136,-126,127
5,s,5.96e-08,1.40e-45,1.18e-38,3.40e+38,24,-149,-126,127
6,d,1.11e-16,4.94e-324,2.23e-308,1.80e+308,53,-1074,-1022,1023
7,q,9.63e-35,0.00e+00,0.00e+00,inf,113,-16494,-16382,16383


### set backend

In [4]:
# pychop.backend('torch')
pychop.backend('numpy', 1) # print information, NumPy is the default option.

Load NumPy backend.


### run chop

In [6]:
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_np)
print("runtime:", time() - st)
print(X_bit[:10, :5])

runtime: 1.601654052734375
[[ 1.7637e+00  4.0015e-01  9.7852e-01  2.2402e+00  1.8672e+00]
 [ 3.0981e-01 -7.3730e-01 -1.5371e+00 -5.6201e-01 -1.5996e+00]
 [-2.0215e-01 -8.3301e-01  1.7334e+00  1.9067e-01 -1.7786e-01]
 [ 2.4707e+00  5.8691e-01 -3.9746e-01  4.3359e-01  5.0049e-01]
 [ 3.3008e-01 -4.7994e-04  8.1836e-01  4.2822e-01 -2.5039e+00]
 [ 3.0200e-01  1.4673e-01  6.1035e-01  1.5518e+00  1.2192e-02]
 [ 3.7134e-01  3.0469e-01  5.0391e-01  1.3525e-01  6.5381e-01]
 [ 3.8721e-01  1.3926e+00 -8.5254e-01 -1.9922e-01 -3.2568e-01]
 [-1.9395e+00  7.3877e-01  1.3945e+00 -8.1348e-01 -8.1885e-01]
 [ 5.6152e-01 -1.3281e+00  4.4952e-02  1.7764e+00  3.6230e-01]]


In [8]:
from pychop.bitchop import bitchop
pychop.backend('numpy')

pyq_f = bitchop(exp_bits=5, sig_bits=10, rmode=1, subnormal=True, random_state=42, device="cpu", verbose=0)
st = time()
X_bit = pyq_f(X_np)
print("runtime:", time() - st)
print(X_bit[:10, :5])

runtime: 69.79691171646118
[[ 1.7637e+00  4.0015e-01  9.7852e-01  2.2402e+00  1.8672e+00]
 [ 3.0957e-01 -7.3730e-01 -1.5371e+00 -5.6201e-01 -1.5996e+00]
 [-2.0215e-01 -8.3301e-01  1.7334e+00  1.9067e-01 -1.7773e-01]
 [ 2.4688e+00  5.8691e-01 -3.9746e-01  4.3359e-01  5.0000e-01]
 [ 3.3008e-01 -4.7994e-04  8.1836e-01  4.2822e-01 -2.5039e+00]
 [ 3.0200e-01  1.4673e-01  6.1035e-01  1.5518e+00  1.2192e-02]
 [ 3.7109e-01  3.0469e-01  5.0391e-01  1.3525e-01  6.5332e-01]
 [ 3.8721e-01  1.3926e+00 -8.5254e-01 -1.9922e-01 -3.2568e-01]
 [-1.9395e+00  7.3877e-01  1.3945e+00 -8.1348e-01 -8.1836e-01]
 [ 5.6152e-01 -1.3281e+00  4.4922e-02  1.7764e+00  3.6230e-01]]


In [ ]:
print("2^"+str(int(np.log2(pyq_f.u))))

In [4]:
pychop.backend('torch', 1) # print information
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit)

Load Troch backend.
runtime: 2.3097269535064697
tensor([[ 1.7637,  0.4001,  0.9785,  ...,  0.9292,  0.2294,  0.4143],
        [ 0.3098, -0.7373, -1.5371,  ...,  0.5171, -0.0329,  1.2979],
        [-0.2021, -0.8330,  1.7334,  ...,  0.7529, -0.5811, -0.1984],
        ...,
        [-0.9595, -1.0127,  0.3489,  ..., -0.3508, -1.2275, -0.9102],
        [ 0.4792,  0.1483,  1.4375,  ...,  0.8120,  1.3428,  0.1327],
        [ 0.3562, -0.3137, -0.6094,  ..., -0.1802, -2.8457, -0.4275]])


In [12]:
pychop.backend('torch', 1) # print information
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit)

Load Troch backend.
runtime: 1.618636131286621
tensor([[ 1.7637,  0.4001,  0.9785,  ...,  0.9292,  0.2294,  0.4143],
        [ 0.3098, -0.7373, -1.5371,  ...,  0.5171, -0.0329,  1.2979],
        [-0.2021, -0.8330,  1.7334,  ...,  0.7529, -0.5811, -0.1984],
        ...,
        [-0.9595, -1.0127,  0.3489,  ..., -0.3508, -1.2275, -0.9102],
        [ 0.4792,  0.1483,  1.4375,  ...,  0.8120,  1.3428,  0.1327],
        [ 0.3562, -0.3137, -0.6094,  ..., -0.1802, -2.8457, -0.4275]])


In [5]:
pychop.backend('torch', 1) # print information
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit)

Load Troch backend.
runtime: 1.661458969116211
tensor([[ 1.7637,  0.4001,  0.9785,  ...,  0.9292,  0.2294,  0.4143],
        [ 0.3098, -0.7373, -1.5371,  ...,  0.5171, -0.0329,  1.2979],
        [-0.2021, -0.8330,  1.7334,  ...,  0.7529, -0.5811, -0.1984],
        ...,
        [-0.9595, -1.0127,  0.3489,  ..., -0.3508, -1.2275, -0.9102],
        [ 0.4792,  0.1483,  1.4375,  ...,  0.8120,  1.3428,  0.1327],
        [ 0.3562, -0.3137, -0.6094,  ..., -0.1802, -2.8457, -0.4275]])


In [5]:
pychop.backend('torch', 1) # print information
pyq_f = chop('h')
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit)

Load Troch backend.
runtime: 1.7425360679626465
tensor([[1.7637, 0.4001, 0.9785,  ..., 0.9292, 0.2294, 0.4143],
        [0.3098, 0.7373, 1.5371,  ..., 0.5171, 0.0329, 1.2979],
        [0.2021, 0.8330, 1.7334,  ..., 0.7529, 0.5811, 0.1984],
        ...,
        [0.9595, 1.0127, 0.3489,  ..., 0.3508, 1.2275, 0.9102],
        [0.4792, 0.1483, 1.4375,  ..., 0.8120, 1.3428, 0.1327],
        [0.3562, 0.3137, 0.6094,  ..., 0.1802, 2.8457, 0.4275]])


In [3]:
pychop.backend('jax', 1) # print information

pyq_f = chop('h')


Load JAX backend.


In [5]:
st = time()
X_bit = pyq_f(X_jx)
print("runtime:", time() - st)
print(X_bit)

runtime: 15.599064111709595
[[ 1.7636719   0.40014648  0.9785156  ...  0.9291992   0.22937012
   0.41430664]
 [ 0.30981445 -0.7373047  -1.5371094  ...  0.51708984 -0.03292847
   1.2978516 ]
 [-0.20214844 -0.8330078   1.7333984  ...  0.7529297  -0.5810547
  -0.19836426]
 ...
 [-0.95947266 -1.0126953   0.34887695 ... -0.35083008 -1.2275391
  -0.91015625]
 [ 0.47924805  0.14831543  1.4375     ...  0.8120117   1.3427734
   0.13269043]
 [ 0.35620117 -0.3137207  -0.609375   ... -0.18017578 -2.8457031
  -0.42749023]]


In [ ]:
pychop.backend('jax', 1) # print information
st = time()
pyq_f = chop('h')
X_bit = pyq_f(X_jx)
print("runtime:", time() - st)
print(X_bit)

In [ ]:
from pychop.chop import chop
pychop.backend('numpy')
pyq_f = chop('h', rmode=5, flip=0)
pyq_f(X_np)

### integer quantization

In [ ]:
pychop.backend('numpy')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_np)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_np)

In [ ]:
pychop.backend('torch')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_th)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_np)

In [ ]:
pychop.backend('jax')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_jx)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_jx)

### fixed point quantization

In [ ]:
pychop.backend('numpy')
pyq_f = pychop.fpoint()

pyq_f(X_np)

In [ ]:
pychop.backend('torch')
pyq_f = pychop.fpoint()
pyq_f(X_th)

In [ ]:
pychop.backend('jax')
pyq_f = pychop.fpoint()
pyq_f(X_jx)